    <크롤링을 통해서 하고 싶은 것>

    1. 타켓팅 선정
        * 리뷰를 데이터를 통해서 상품을 사용한 고객군 파악 가능
            - 연령층
            - 피부고민
            - 피부타입
        * 이지듀 공식 홈페이지에서 리뷰가 많은 상품별 target 연령층, 피부고민, 피부타입 등으로 코호트를 나눌 수 있음
        * 연령층을 바탕으로 고객의 특징을 뽑아 카피를 만들 수 있음 (ex. 30대 뭐시기, 40대 뭐시기~)
            - 연령층을 본 이후는 나중에 고객이 광고를 들어왔을 때 리뷰를 볼 수 있는데,
              리뷰를 작성한 나이대를 확인할 수 있음
            - 그런데 20대가 들어왔는데 30대, 40대 리뷰가 있다면 괴리감을 느낄 수 있을 것으로 보임

    2. 리뷰를 통해 카피로 쓸 키워드 뽑아내기
        * 리뷰를 직접 읽는 것이 좋을 수도 있음
        * 그런데 키워드를 뽑아내서 카피를 뽑아낼 수 있을 것으로 보임
            - why? 고객이 직접 말한 것을 토대로 카피를 작성하는 것이 신뢰도가 있을 것?으로 보임 (데이터 기반)

    
    <고려해야 할 사항>
    
    * 왜 리뷰를 통해서 봤는지 설명해야 할까?
        - 업계 분석을 통해서, 이지듀가 강점이 있는 상품들을 통해 유입을 시키려고 했다고 설명해야 할까?
        - why 업계분석? to find trend
        - 리뷰가 많은 상품 --> 만족도가 높을 가능성이 있는 상품
        - 이를 통해 우선
    * customer acquistion cost 확인하기
        - 유입을 시키는 것도 중요하지만, 실제 유입이 되서 물건을 구매해야 함
        - 때문에 현재 리뷰를 통해서 상품을 구매한 고객들을 기반으로 타겟팅을 선정하고자 함.
    * 상품 구매와 멤버십 구매
        - 상품 구매를 통해 공식 홈페이지에서의
    * 시각화는 plotly로 깔끔하게

## Crawling Function

In [ ]:
# 한국어 설정
# 런타임 재시작 필요
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
# libraries
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

%matplotlib inline

## Product ID

In [ ]:
# products id

# 새로운 상품과 할인 상품을 통해 유입을 더 높이기? + 매출 증대에 영향

# -- 미백/기미 --
# EGF 기미엠플: 2073 (one of the products with a number of reviews)
# 기미크림: 2226 (one of the products with a number of reviews)
# 이지듀 EX DW-EGF 멜라토닝 패치 8매 (기미앰플 1ml 증정): 2079 (one of the products with a number of reviews)
# [~66%] 기미 토닝 2700샷: 2221

# -- 트러블/민감/진정 --
# [NEW 손상완화크림] 베리어 리페어 크림: 2227
# [런칭특가 50%] EGF 흔적세럼: 2303
# [런칭특가 50%] 아크네 클리어 SOS 스팟: 2288
# [런칭특가 ~62% ] EGF 필링세럼 + 해면: 2314

# -- 모공/탄력/주름 --
# DW-EGF 크림 프레좀 50ml: 1646
# 이지듀 DW-EGF 더마 톡스 니들샷: 2185

In [2]:
# functions

def review_crawler(product_code: str, page_num: int):
    final_review_list = []
    for num in tqdm(range(1, page_num+1)):
        url = 'https://review9.cre.ma/easydew.co.kr/products/reviews?product_code={}&app=0&iframe=1&iframe_id=crema-product-reviews-2&page={}'.format(product_code, str(num))
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            reviews = soup.select('div.review_list_v2__review_rcontent')
        else :
            print('status code --> ', response.status_code)

        review_list = []
        for review in reviews:
            reviewer_name = review.select_one('div.review_list_v2__user_name_message')
            review_types = review.select('span.review_options_v2__name')
            review_values = review.select('span.review_options_v2__value')
            review_dict = {}
            for review_type, review_val in zip(review_types, review_values):
                review_dict['reviewer'] = reviewer_name.text.split('님')[0].strip()
                review_dict[review_type.text.strip()] = review_val.text.strip()

            if review_dict != {}: # review_dict가 비어있지 않을 때만 리스트에 넣어줌 (비어있는 경우가 있기 때문에)
                review_list.append(review_dict)

        final_review_list += review_list

    review_df = pd.DataFrame(final_review_list)
    # review_df = review_df.iloc[:, :-1]
    return review_df

## 미백/기미

> 기미크림

In [4]:
# crawling data
review_df = review_crawler('2226', 100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
# removing useless columns
review_df = review_df.iloc[:, :4+1] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df))

# preprocessing
review_df = review_df.dropna(subset=['연령대'])
review_df['연령대'] = review_df['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df.head(4)

257


,reviewer,피부타입,연령대,피부고민,피부과 관리
0,최****,중성,40대,홍조,경험 없음
1,장****,복합성,20대,트러블,경험 있음
2,유****,지성,50대,기미/잡티,경험 없음
3,박****,건성,50대,주름/영양,경험 있음


In [6]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['grey', 'royalblue', 'grey', 'royalblue'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['royalblue','grey', 'royalblue', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[2] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['grey', 'royalblue'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "기미크림",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> EGF 기미앰플

In [7]:
# crawling data
review_df_1 = review_crawler('2073', 100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
# removing useless columns
review_df_1 = review_df_1.iloc[:, :4+1] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_1))

# preprocessing
review_df_1 = review_df_1.dropna(subset=['연령대'])
review_df_1['연령대'] = review_df_1['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_1.head(4)

378


,reviewer,피부타입,연령대,피부고민,피부과 관리
0,박****,중성,40대,기미/잡티,경험 있음
1,박****,복합성,50대,기미/잡티,경험 없음
2,이****,중성,40대,기미/잡티,경험 있음
3,정****,건성,40대,기미/잡티,경험 있음


In [9]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_1['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['grey', 'royalblue', 'royalblue', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_1['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['royalblue','royalblue', 'grey', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*10
marker_color[0] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_1['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_1['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "EGF 기미앰플",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> 멜라토닝 패치

In [10]:
# crawling data
review_df_2 = review_crawler('2079', 100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
# removing useless columns
review_df_2 = review_df_2.iloc[:, :4+1] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_2))

# preprocessing
review_df_2 = review_df_2.dropna(subset=['연령대'])
review_df_2['연령대'] = review_df_2['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_2.head(4)

225


,reviewer,피부타입,연령대,피부고민,피부과 관리
0,김****,복합성,50대,기미/잡티,경험 있음
1,김****,복합성,40대,화이트닝,NaN
2,박****,복합성,40대,기미/잡티,경험 있음
3,조****,복합성,40대,화이트닝,NaN


In [12]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_2['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['royalblue', 'royalblue', 'grey', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_2['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['royalblue','royalblue', 'grey', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[0] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_2['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_2['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> [~66%] 기미 토닝 2700샷

In [13]:
# crawling data
review_df_3 = review_crawler('2221', 100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
# removing useless columns
review_df_3 = review_df_3.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_3))

# preprocessing
review_df_3 = review_df_3.dropna(subset=['연령대'])
review_df_3['연령대'] = review_df_3['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_3.head(4)

406


,reviewer,피부타입,연령대,피부고민,피부과 관리,기미 토닝 2700샷,추가구성,1개월 관리,[단 하루 66%],[BEST] 더마니들 4개입
0,윤****,지성,30대,트러블,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN
1,최****,건성,40대,기미/잡티,경험 있음,기미앰플(1ml) 4개입 + 더마니들 4개입 + 롤러,NaN,NaN,NaN,NaN
2,김****,복합성,40대,모공/피지,경험 있음,기미앰플(30ml) 대용량 + 더마니들 8개입 + 롤러,NaN,NaN,NaN,NaN
3,김****,건성,50대,기미/잡티,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN


In [15]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_3['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['grey', 'royalblue', 'royalblue', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_3['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['grey','royalblue', 'royalblue', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*10
marker_color[1] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_3['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_3['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "기미 토닝 2700샷",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

## 트러블/민감/진정

In [ ]:
# [NEW 손상완화크림] 베리어 리페어 크림: 2227
# [런칭특가 50%] EGF 흔적세럼: 2303
# [런칭특가 50%] 아크네 클리어 SOS 스팟: 2288
# [런칭특가 ~62% ] EGF 필링세럼 + 해면: 2314

> 베리어 리페어 크림

In [16]:
# crawling data
review_df_4 = review_crawler('2227', 100)
# removing useless columns
review_df_4 = review_df_4.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_4))

# preprocessing
review_df_4 = review_df_4.dropna(subset=['연령대'])
review_df_4['연령대'] = review_df_4['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_4.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

265


,reviewer,피부타입,연령대,피부고민,피부과 관리,[54%] 손상회복크림,모이스트 크림,[~50%] 손상회복크림,손상회복크림,멜라토닝 앰플
0,송****,건성,50대,기미/잡티,경험 있음,[50%] 손상회복크림 1개,NaN,NaN,NaN,NaN
1,박****,건성,40대,민감성,경험 있음,NaN,[39%] 크림 모이스트 3개,NaN,NaN,NaN
2,오****,복합성,20대,민감성,경험 없음,NaN,[20%] 크림 모이스트 1개,NaN,NaN,NaN
3,박****,복합성,20대,민감성,경험 없음,NaN,NaN,NaN,NaN,NaN


In [17]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_4['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['royalblue', 'royalblue', 'grey', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_4['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['royalblue','royalblue', 'crimson', 'crimson', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*10
marker_color[0] = 'royalblue'
marker_color[3] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_4['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_4['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "기미 토닝 2700샷",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> EGF 흔적세럼

In [18]:
# crawling data
review_df_5 = review_crawler('2303', 100)
# removing useless columns
review_df_5 = review_df_5.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_5))

# preprocessing
review_df_5 = review_df_5.dropna(subset=['연령대'])
review_df_5['연령대'] = review_df_5['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_5.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

338


,reviewer,피부타입,연령대,피부고민,피부과 관리,EGF 흔적세럼,트러블 라인,흔적세럼,추가 구성 상품
0,채엽,복합성,20대,트러블,경험 있음,NaN,NaN,NaN,NaN
1,김****,복합성,30대,트러블,경험 있음,NaN,NaN,NaN,NaN
2,장****,복합성,50대,트러블,경험 없음,[50%] 흔적라인 풀세트 [증정] 흔적미백패치 4매,NaN,NaN,NaN
3,박****,복합성,30대,트러블,경험 있음,[50%] 아크네 클리어 흔적세럼,NaN,NaN,NaN


In [19]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_5['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['royalblue', 'royalblue', 'grey', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_5['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['crimson','crimson', 'royalblue', 'royalblue', 'crimson'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[0] = 'royalblue'
marker_color[2] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_5['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_5['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "EGF 흔적 세럼",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> 아크네 클리어 SOS 스팟

In [20]:
# crawling data
review_df_6 = review_crawler('2288', 100)
# removing useless columns
review_df_6 = review_df_6.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_6))

# preprocessing
review_df_6 = review_df_6.dropna(subset=['연령대'])
review_df_6['연령대'] = review_df_6['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_6.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

423


,reviewer,피부타입,연령대,피부고민,피부과 관리,트러블 라인,추가 구성 상품
0,김****,복합성,40대,기미/잡티,경험 있음,NaN,NaN
1,채엽,복합성,20대,트러블,경험 있음,NaN,NaN
2,안****,복합성,50대,홍조,경험 있음,NaN,NaN
3,김****,복합성,30대,트러블,경험 있음,NaN,NaN


In [21]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_6['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['royalblue', 'grey', 'royalblue', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_6['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['royalblue','crimson', 'royalblue', 'crimson', 'crimson'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[1] = 'royalblue'
marker_color[0] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_6['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_6['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "아크네 클리어 SOS 스팟",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> EGF 필링세럼 + 해면

In [22]:
# crawling data
review_df_7 = review_crawler('2314', 20)
# removing useless columns
review_df_7 = review_df_7.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_7))

# preprocessing
review_df_7 = review_df_7.dropna(subset=['연령대'])
review_df_7['연령대'] = review_df_7['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_7.head(4)

  0%|          | 0/20 [00:00<?, ?it/s]

83


,reviewer,피부타입,연령대,피부고민,피부과 관리,필링세럼,추가구성상품
0,윤****,지성,30대,트러블,경험 있음,[61%] 필링세럼 4개,NaN
1,유****,복합성,40대,트러블,경험 있음,[61%] 필링세럼 4개,NaN
2,현****,복합성,30대,트러블,경험 있음,[50%] 필링세럼 1개,NaN
3,장****,복합성,50대,트러블,경험 없음,NaN,[60%] 아크네 클리어 모공청소토너


In [23]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_7['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['royalblue', 'royalblue', 'grey', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_7['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['crimson','royalblue', 'royalblue', 'crimson', 'crimson'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[0] = 'royalblue'
marker_color[2] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_7['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_7['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "EGF 필링세럼 + 해면",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

## 모공/탄력/주름

In [ ]:
# DW-EGF 크림 프레좀 50ml: 1646
# 이지듀 DW-EGF 더마 톡스 니들샷: 2185

> DW-EGF 크림 프레좀 50ml

In [24]:
# crawling data
review_df_8 = review_crawler('1646', 100)
# removing useless columns
review_df_8 = review_df_8.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_8))

# preprocessing
review_df_8 = review_df_8.dropna(subset=['연령대'])
review_df_8['연령대'] = review_df_8['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_8.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

123


,reviewer,피부타입,연령대,피부고민,프레좀,피부과 관리,[37%] 프레좀 탄력크림,[45%] 프레좀 탄력크림,프레좀 탄력크림
0,곽****,중성,30대,민감성,[20%] 크림 프레좀 2개,NaN,NaN,NaN,NaN
1,이****,건성,50대,주름/영양,[10%] 크림 프레좀 1개,경험 있음,NaN,NaN,NaN
2,강****,건성,40대,주름/영양,[10%] 크림 프레좀 1개,경험 없음,NaN,NaN,NaN
3,나****,복합성,20대,민감성,[10%] 크림 프레좀 1개,NaN,NaN,NaN,NaN


In [25]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_8['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['grey', 'royalblue', 'royalblue', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_8['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['crimson','royalblue', 'royalblue', 'crimson', 'royalblue'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[1] = 'royalblue'
marker_color[3] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_8['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_8['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "DW-EGF 크림 프레좀 50ml",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

> 이지듀 DW-EGF 더마 톡스 니들샷

In [26]:
# crawling data
review_df_9 = review_crawler('2185', 100)
# removing useless columns
review_df_9 = review_df_9.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_9))

# preprocessing
review_df_9 = review_df_9.dropna(subset=['연령대'])
review_df_9['연령대'] = review_df_9['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_9.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

411


,reviewer,피부타입,연령대,피부고민,피부과 관리,기미 토닝 2700샷,더마니들(롤러미포함),추가구성,1개월 관리,[단 하루 66%]
0,윤****,지성,30대,트러블,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN
1,최****,건성,40대,기미/잡티,경험 있음,기미앰플(1ml) 4개입 + 더마니들 4개입 + 롤러,NaN,NaN,NaN,NaN
2,김****,복합성,40대,모공/피지,경험 있음,기미앰플(30ml) 대용량 + 더마니들 8개입 + 롤러,NaN,NaN,NaN,NaN
3,김****,건성,50대,기미/잡티,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN


In [27]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=4,
                    subplot_titles=('<피부타입>', '<연령대>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)
# 피부타입
fig.add_trace(go.Histogram(
    x=review_df_9['피부타입']
), row=1, col=1)
fig.update_traces(marker_color=['grey', 'royalblue', 'royalblue', 'grey'],
                  row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)

# 연령대
fig.add_trace(go.Histogram(
    x=review_df_9['연령대']
), row=1, col=2)
fig.update_traces(marker_color=['crimson','royalblue', 'royalblue', 'grey', 'grey'], row=1, col=2)

# 피부고민
# marker color for 피부고민
marker_color=['grey']*8
marker_color[1] = 'royalblue'
# marker_color[3] = 'royalblue'
# marker_color[4] = 'royalblue'

fig.add_trace(go.Histogram(
    x=review_df_9['피부고민']
), row=1, col=3)
fig.update_traces(marker_color=marker_color, row=1, col=3)

# 피부과 관리
fig.add_trace(go.Histogram(
    x=review_df_9['피부과 관리'],
), row=1, col=4)
fig.update_traces(marker_color=['royalblue', 'grey'], xbins_size=0.1,
                  row=1, col=4)

fig.update_layout(height=450, width=1200,
                  title={'text': "이지듀 DW-EGF 더마 톡스 니들샷",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  showlegend=False
                  )
# fig.update_yaxes(visible=False)
fig.show()

## Cohort Analysis Base

In [429]:
# Dashboard

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
age_list = list(sorted(review_df['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='green',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='yellowgreen',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='green',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='yellowgreen',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='green',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='yellowgreen',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령별 이지듀 DW-EGF 더마 톡스 니들샷",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()